# Saliva Analysis

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from stresspose_analysis.datasets.mainstudy import MainStudyDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
root_path = Path("../../..")

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))
base_path = Path(config_dict[deploy_type]["base_path"])
dataset = MainStudyDataset(base_path)
dataset

In [ ]:
output_path = root_path.joinpath("results")
img_path = output_path.joinpath("plots")
stats_path = output_path.joinpath("statistics")

bp.utils.file_handling.mkdirs([img_path, stats_path])

## Prepare Data

In [ ]:
# dict for renaming condition names for use in plots
condition_mapping = {"tsst": "TSST", "ftsst": "f-TSST"}

In [ ]:
cort_samples = dataset.add_cortisol_index(dataset.cortisol)
# rename condition names for nicer plots
cort_samples = cort_samples.rename(condition_mapping)
cort_samples.head()

In [ ]:
cort_features = dataset.add_cortisol_index(dataset.cortisol_features)
# rename condition names for nicer plots
cort_features = cort_features.rename(condition_mapping)
cort_features.head()

In [ ]:
tsst = bp.protocols.TSST(saliva_plot_params={"test_title": "(f-)TSST"})
tsst.add_saliva_data(cort_samples, "cortisol", dataset.sample_times_saliva, sample_times_absolute=True)

In [ ]:
dict_information = {}

## Descriptive Information

In [ ]:
cort_increase = cort_features.xs("max_inc_percent", level="saliva_feature")
cort_increase_mean = cort_increase.groupby("condition").agg(["mean", "std"])
dict_information["cortisol_increase"] = cort_increase_mean

cort_increase_mean

## Cortisol Samples

### Plots

In [ ]:
hue_order = ["f-TSST", "TSST"]

In [ ]:
# cort_samples_analysis = cort_samples.reindex([1, 3], level="sample")
cort_pre = cort_samples.xs("S1", level="sample")
cort_post = pd.DataFrame(
    cort_samples.reindex(["S3", "S4"], level="sample").unstack("sample").max(axis=1), columns=["cortisol"]
)
cort_samples_analysis = pd.concat({"pre": cort_pre, "post": cort_post}, names=["sample"])

steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {
    "dv": "cortisol",
    "within": "sample",
    "subject": "subject",
    "groupby": "condition",
    "parametric": False,
    "multicomp": {"method": "bonf", "levels": True},
}

sample_pipeline = bp.stats.StatsPipeline(steps, params)
sample_pipeline.apply(cort_samples_analysis)
sample_pipeline.display_results()
sample_pipeline.export_statistics(stats_path.joinpath("stats_cortisol_pre_post.xlsx"))

In [ ]:
fig, ax = plt.subplots()

tsst.saliva_plot("cortisol", hue_order=hue_order, ax=ax)
fig.tight_layout()

fig.savefig(img_path.joinpath("img_cortisol_response_mainstudy.pdf"), transparent=True)

## Cortisol Features

### Statistics

In [ ]:
cort_features_analysis = cort_features.reindex(["auc_g", "max_inc"], level="saliva_feature")

steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {
    "dv": "cortisol",
    "within": "condition",
    "subject": "subject",
    "groupby": "saliva_feature",
    "parametric": False,
    "multicomp": {"method": "bonf", "levels": True},
}

feature_pipeline = bp.stats.StatsPipeline(steps, params)
feature_pipeline.apply(cort_features_analysis)
feature_pipeline.export_statistics(stats_path.joinpath("stats_cortisol_response.xlsx"))
feature_pipeline.display_results()

### Plots

In [ ]:
features = ["max_inc", "auc_g"]

box_pairs, pvalues = feature_pipeline.sig_brackets(
    "test", stats_effect_type="within", plot_type="multi", features=features, x="condition", subplots=True
)

fig, axs = bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=cort_features_analysis,
    saliva_type="cortisol",
    features=features,
    x="saliva_feature",
    hue="condition",
    hue_order=hue_order,
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    legend_loc="upper center",
    legend_orientation="horizontal",
    rect=(0, 0, 1, 0.925),
    palette=cmaps.faculties_light,
)

## Multiplot – Cortisol Response + Boxplots

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(12, 4), gridspec_kw={"width_ratios": [4, 1, 1]})

features = ["max_inc", "auc_g"]
xticklabels = {"max_inc": ["f-TSST", "TSST"], "auc_g": ["f-TSST", "TSST"]}

tsst.saliva_plot("cortisol", hue_order=hue_order, ax=axs[0])


box_pairs, pvalues = feature_pipeline.sig_brackets(
    "test", stats_effect_type="within", plot_type="single", features=features, x="condition", subplots=True
)

bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=cort_features_analysis,
    saliva_type="cortisol",
    x="condition",
    features=features,
    xticklabels=xticklabels,
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    legend_loc="upper right",
    legend_orientation="vertical",
    palette=cmaps.faculties_light,
    axs=axs[1:],
)

for ax, text, title in zip(axs, list("123"), ["Cortisol Response to (f-)TSST", r"$\Delta c_{max}$", r"$AUC_{G}$"]):
    ax.set_title(f"({text}) {title}")

for ax in axs[1:]:
    ax.set_xlabel("Condition")

fig.tight_layout(w_pad=1.0)
fig.savefig(img_path.joinpath("img_cortisol_response_multiplot_mainstudy.pdf"), transparent=True)

## TeX Output

In [ ]:
sample_pipeline.stats_to_latex(stats_test="pairwise_tests")

In [ ]:
feature_pipeline.stats_to_latex(stats_test="pairwise_tests")

## Export

In [ ]:
bp.io.write_pandas_dict_excel(dict_information, output_path.joinpath("descriptive_information_saliva.xlsx"))